In [5]:
##### import pandas as pd
import numpy as np
import pandas as pd
import json
from sklearn import model_selection
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold, cross_val_score
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

# À voir

# from imports import data_import # Fichier de fonctions utiles à l'import des données


pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 380)

In [2]:
# Imports des jeux de données
# df = data_import() # Renvoie un dataframe comprenant la fusion des résultats des matchs et des cotes des bookmakers

# dataframe des données saisons
data_saison = json.load(open("season_match_stats.json"))

table = []

for index, value in data_saison.items():
    ligne = [index] + list(value.values())  
    table.append(ligne)
    
df_saison = pd.DataFrame(table, columns = ['id_match', 'home_team_id', 'away_team_id', 'home_team_name', 'away_team_name', 'date_string', 'half_time_score', 'full_time_score'])
df_saison.head()



,id_match,home_team_id,away_team_id,home_team_name,away_team_name,date_string,half_time_score,full_time_score
0,1080506,13,26,Arsenal,Liverpool,14/08/2016 16:00:00,1 : 1,3 : 4
1,1080507,183,32,Bournemouth,Manchester United,14/08/2016 13:30:00,0 : 1,1 : 3
2,1080508,184,259,Burnley,Swansea,13/08/2016 15:00:00,0 : 0,0 : 1
3,1080509,15,29,Chelsea,West Ham,15/08/2016 20:00:00,0 : 0,2 : 1
4,1080510,162,175,Crystal Palace,West Bromwich Albion,13/08/2016 15:00:00,0 : 0,0 : 1


In [3]:
#dataframe des données 

data_match = json.load(open("season_stats.json",encoding='utf-8'))

table = []

for index, value in data_match.items():
    home = dict(list(value.values())[0])
    ligne = [index] + list(home['team_details'].values())  
    table.append(ligne)
    
df_match_home = pd.DataFrame(table, columns = ['id_match', 'team_id_home', 'team_name_home', 'team_rating_home', 'date'])
compteur = 0

for index, value in data_match.items():
    home = dict(list(value.values())[0])
    
    for i,j in home['aggregate_stats'].items() :
        df_match_home.loc[compteur, i] = j
         
    compteur += 1
    
print("dataframe des equipes à domicile :")    
df_match_home.head()

table = []

for index, value in data_match.items():
    away = dict(list(value.values())[1])
    ligne = [index] + list(away['team_details'].values())  
    table.append(ligne)
    
df_match_away = pd.DataFrame(table, columns = ['id_match', 'team_id_away', 'team_name_away', 'team_rating_away', 'date'])

compteur = 0
for index, value in data_match.items():
    away = dict(list(value.values())[1])
    
    for i,j in away['aggregate_stats'].items() :
        df_match_away.loc[compteur, i] = j
         
    compteur += 1
    
print("dataframe des equipes à l'extérieur :")    
df_match_away.head()

dataframe des equipes à domicile :
dataframe des equipes à l'extérieur :


,id_match,team_id_away,team_name_away,team_rating_away,date,won_contest,possession_percentage,total_throws,blocked_scoring_att,penalty_save,total_scoring_att,att_sv_low_left,total_tackle,aerial_won,aerial_lost,accurate_pass,total_pass,won_corners,shot_off_target,ontarget_scoring_att,goals,att_miss_left,fk_foul_lost,total_offside,att_sv_low_right,att_goal_high_left,att_goal_low_centre,att_sv_high_left,att_sv_low_centre,att_goal_low_left,att_miss_right,att_goal_low_right,att_miss_high_left,att_miss_high_right,att_miss_high,att_sv_high_centre,att_pen_goal,att_goal_high_right,att_sv_high_right,att_goal_high_centre,att_post_right,post_scoring_att,att_post_high,att_post_left
0,1080506,26,Liverpool,7.02714285714286,14/08/2016,11,49.3,22,6,1,16,1,21,17,22,380,479,4,3,7,4,3,17,3,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1080507,32,Manchester United,7.05428571428571,14/08/2016,8,52.6,17,2,NaN,11,NaN,27,8,8,488,561,2,2,7,3,NaN,10,4,NaN,NaN,NaN,1,3,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1080508,259,Swansea,7.32071428571429,13/08/2016,8,52.5,13,2,NaN,17,2,19,32,20,297,383,4,6,9,1,1,14,2,2,NaN,NaN,1,2,NaN,1,1,1,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1080509,29,West Ham,6.58214285714286,15/08/2016,3,38.3,12,2,NaN,7,1,25,25,24,257,350,1,2,3,1,1,16,1,NaN,NaN,NaN,NaN,1,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1080510,175,West Bromwich Albion,7.03384615384615,13/08/2016,5,37.7,34,2,NaN,13,NaN,18,21,21,143,245,6,8,3,1,3,15,2,1,NaN,NaN,NaN,1,1,2,NaN,1,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#merge des df domicile et extérieur

df_1617 = pd.merge(df_match_home,df_match_away,how='left',on='id_match')
df_1617.head()

df_1617 = df_1617.fillna(0)
df_1617 = df_1617.sort_values(by='id_match',ascending=True)
df_1617.head()

# Changement des types des colonnes 
cols=[i for i in df_1617.columns if i not in ["team_name_home","team_rating_home","date_x",'possession_percentage_x','team_name_away','team_rating_away','date_y','possession_percentage_y']]
for column in cols:
    df_1617[column] = df_1617[column].astype(int)

cols=[i for i in df_1617.columns if i in ["team_name_home","date_x",'team_name_away','date_y']]
for column in cols:
    df_1617[column] = df_1617[column].apply(str)
    
cols=[i for i in df_1617.columns if i in ["team_rating_home",'possession_percentage_x','team_rating_away','possession_percentage_y']]
for column in cols:
    df_1617[column] = df_1617[column].apply(float)
    
df_1617.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 380 entries, 0 to 379
Data columns (total 87 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id_match                 380 non-null    int32  
 1   team_id_home             380 non-null    int32  
 2   team_name_home           380 non-null    object 
 3   team_rating_home         380 non-null    float64
 4   date_x                   380 non-null    object 
 5   att_goal_low_right_x     380 non-null    int32  
 6   won_contest_x            380 non-null    int32  
 7   possession_percentage_x  380 non-null    float64
 8   total_throws_x           380 non-null    int32  
 9   blocked_scoring_att_x    380 non-null    int32  
 10  att_sv_low_left_x        380 non-null    int32  
 11  total_tackle_x           380 non-null    int32  
 12  aerial_won_x             380 non-null    int32  
 13  att_miss_right_x         380 non-null    int32  
 14  att_sv_high_centre_x     3

In [5]:
#affichage du score

df_score=df_1617[['id_match','team_name_home','goals_x','goals_y','team_name_away']]
df_score.head()

df_score.insert(5,'winner',np.linspace(0,0,len(df_score['team_name_home'])))


# Ajout d'une colonne winner
for i in df_score['team_name_home'].index:
    
    if(df_score['goals_x'][i] > df_score['goals_y'][i]):
        df_score['winner'][i]=0#df_score['team_name_home'][i]  :  équivaut à 
    elif(df_score['goals_x'][i]  < df_score['goals_y'][i]):
        df_score['winner'][i]=1#df_score['team_name_away'][i]
    else:
        df_score['winner'][i]=2#'draw'

<ipython-input-5-ac8da0a13900>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_score['winner'][i]=1#df_score['team_name_away'][i]
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-5-ac8da0a13900>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_score['winner'][i]=0#df_score['team_name_h

In [6]:
#ajout de la colonne winner sur df_1617
df_1617 = pd.merge(df_1617,df_score[['id_match','winner']], how='left', on='id_match')
df_1617.head()

# on dichotomise la variable winner
df_1617 = df_1617.join(pd.get_dummies(df_score['winner']))
df_1617 = df_1617.drop('winner', axis = 1)
df_1617.head()

df_1617 = df_1617.rename({0.0: 'Home_wins', 1.0:'Away_wins' , 2.0 : 'Draw'}, axis = 1)
df_1617.head()

#renomme la colonne date_x et suppression de la colonne date_y
df_1617 = df_1617.rename({'date_x':'date'}, axis = 1)
df_1617 = df_1617.drop('date_y', axis = 1)

In [7]:
#réorganisation des colonnes
cols = list(df_1617.columns)
cols

#réorganisation des colonnes en pairs (home à gauche et away à droite) pour faciliter la comparaison
cols_1 = ['id_match',
 'date',
 'team_id_home',
 'team_name_home',
 'team_rating_home',
 'team_id_away',
 'team_name_away',
 'team_rating_away',
 'Home_wins',
 'Draw',
 'Away_wins'
]


cols_2 =  ['accurate_pass_x',
 'accurate_pass_y',
 'blocked_scoring_att_x',
 'blocked_scoring_att_y',
 'goals_x',
 'goals_y',
 'ontarget_scoring_att_x',
 'ontarget_scoring_att_y',
 'possession_percentage_x',
 'possession_percentage_y',
 'post_scoring_att_x',
 'post_scoring_att_y',
 'shot_off_target_x',
 'shot_off_target_y',
 'won_contest_x',
 'won_contest_y',
 'won_corners_x',
 'won_corners_y']

cols_2.sort()

df_1617 = df_1617[cols_1 + cols_2]
df_1617.head()

,id_match,date,team_id_home,team_name_home,team_rating_home,team_id_away,team_name_away,team_rating_away,Home_wins,Draw,Away_wins,accurate_pass_x,accurate_pass_y,blocked_scoring_att_x,blocked_scoring_att_y,goals_x,goals_y,ontarget_scoring_att_x,ontarget_scoring_att_y,possession_percentage_x,possession_percentage_y,post_scoring_att_x,post_scoring_att_y,shot_off_target_x,shot_off_target_y,won_contest_x,won_contest_y,won_corners_x,won_corners_y
0,1080506,14/08/2016,13,Arsenal,6.677857,26,Liverpool,7.027143,0,0,1,386,380,1,6,3,4,5,7,50.7,49.3,0,0,3,3,10,11,5,4
1,1080507,14/08/2016,183,Bournemouth,6.337857,32,Manchester United,7.054286,0,0,1,412,488,4,2,1,3,3,7,47.4,52.6,0,0,2,2,13,8,4,2
2,1080508,13/08/2016,184,Burnley,6.590000,259,Swansea,7.320714,0,0,1,249,297,2,2,0,1,3,9,47.5,52.5,0,0,5,6,6,8,7,4
3,1080509,15/08/2016,15,Chelsea,6.901429,29,West Ham,6.582143,1,0,0,480,257,4,2,2,1,6,3,61.7,38.3,0,0,6,2,16,3,7,1
4,1080510,13/08/2016,162,Crystal Palace,6.670714,175,West Bromwich Albion,7.033846,0,0,1,298,143,3,2,0,1,4,3,62.3,37.7,0,0,7,8,8,5,3,6


In [8]:
# Choix des variables à retenir
# Dichotomisation des variables catégorielles

In [9]:
#import bookmakers
bookmakers = pd.read_csv('season-1617_bookmakers.csv')
bookmakers = bookmakers.rename({'Date': 'date', 'HomeTeam':'team_name_home', 'AwayTeam' : 'team_name_away'}, axis = 1)
bookmakers = bookmakers.drop(['Div', 'FTHG', 'FTAG', 'HTHG', 'HTR', 'Referee'], axis = 1)
bookmakers.head()

,date,team_name_home,team_name_away,FTR,HTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,Bb1X2,BbMxH,BbAvH,BbMxD,BbAvD,BbMxA,BbAvA,BbOU,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,13/08/16,Burnley,Swansea,A,0,10,17,3,9,10,14,7,4,3,2,0,0,2.40,3.3,3.25,2.45,3.1,2.95,2.50,3.3,2.65,2.45,3.25,3.10,2.47,3.32,3.19,2.50,3.20,2.90,2.50,3.2,3.25,55,2.55,2.43,3.35,3.21,3.30,3.10,40,2.40,2.30,1.68,1.61,32,-0.25,2.13,2.06,1.86,1.81,2.79,3.16,2.89
1,13/08/16,Crystal Palace,West Brom,A,0,14,13,4,3,12,15,3,6,2,2,0,0,2.00,3.3,4.50,2.00,3.2,3.90,2.10,3.3,3.30,2.00,3.25,4.33,2.06,3.29,4.32,2.05,3.10,4.00,2.00,3.3,4.40,56,2.10,2.01,3.40,3.23,4.50,4.16,38,2.68,2.50,1.60,1.52,33,-0.50,2.07,2.00,1.90,1.85,2.25,3.15,3.86
2,13/08/16,Everton,Tottenham,D,0,12,13,6,4,10,14,5,6,0,0,0,0,3.20,3.4,2.40,2.95,3.2,2.40,2.65,3.3,2.50,3.10,3.40,2.40,3.25,3.43,2.37,3.10,3.10,2.40,3.25,3.4,2.38,55,3.30,3.12,3.45,3.32,2.50,2.36,41,2.12,2.05,1.87,1.77,32,0.25,1.91,1.85,2.09,2.00,3.64,3.54,2.16
3,13/08/16,Hull,Leicester,H,0,14,18,5,5,8,17,5,3,2,2,0,0,4.50,3.6,1.91,4.33,3.4,1.90,3.30,3.3,2.10,4.50,3.50,1.91,4.43,3.55,1.95,4.20,3.25,1.95,4.40,3.5,1.95,55,4.50,4.17,3.60,3.43,2.33,1.95,40,2.30,2.19,1.74,1.67,31,0.25,2.35,2.26,2.03,1.67,4.68,3.50,1.92
4,13/08/16,Man City,Sunderland,H,0,16,7,4,3,11,14,9,6,1,2,0,0,1.25,6.5,15.00,1.22,6.0,11.50,1.25,5.5,10.30,1.25,6.50,13.00,1.27,6.48,13.15,1.25,5.50,13.00,1.25,6.5,15.00,56,1.30,1.25,6.80,6.11,15.00,12.55,39,1.56,1.53,2.67,2.48,34,-1.50,1.81,1.73,2.20,2.14,1.25,6.50,14.50


In [10]:
column_odds = ['B365H', 'B365D', 'B365A',
       'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH',
       'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'PSCH', 'PSCD', 'PSCA']

In [11]:
home_odds = [l for l in column_odds if l[-1] == 'H']
draw_odds = [l for l in column_odds if l[-1] == 'D']
away_odds = [l for l in column_odds if l[-1] == 'A']
print(home_odds, draw_odds, away_odds)

['B365H', 'BWH', 'IWH', 'LBH', 'PSH', 'WHH', 'VCH', 'PSCH'] ['B365D', 'BWD', 'IWD', 'LBD', 'PSD', 'WHD', 'VCD', 'PSCD'] ['B365A', 'BWA', 'IWA', 'LBA', 'PSA', 'WHA', 'VCA', 'PSCA']


In [12]:
#détermination des meilleurs bookmakers
bookmakers_home = bookmakers[home_odds].idxmax(axis = 1).value_counts()
bookmakers_home = bookmakers_home.rename(lambda x : x[:-1])
bookmakers_draw = bookmakers[draw_odds].idxmax(axis = 1).value_counts()
bookmakers_draw = bookmakers_draw.rename(lambda x : x[:-1])
bookmakers_away = bookmakers[away_odds].idxmax(axis = 1).value_counts()
bookmakers_away = bookmakers_away.rename(lambda x : x[:-1])
bookmakers_best = pd.Series([bookmakers_home,bookmakers_draw,bookmakers_away]).sum()
bookmakers_best = bookmakers_best.fillna(0)
print(bookmakers_best)
;

B365    209
BW       13
IW      146
LB       15
PS      227
PSC     426
VC       63
WH       41
dtype: int64


''

In [13]:
bookmakers_home_max = bookmakers[home_odds].max(axis=1)
bookmakers_draw_max = bookmakers[draw_odds].max(axis=1)
bookmakers_away_max = bookmakers[away_odds].max(axis=1)
bookmakers_max = pd.concat([bookmakers[['date','team_name_home','team_name_away', 'FTR']], bookmakers_home_max, bookmakers_draw_max,
                            bookmakers_away_max], axis = 1)
bookmakers_max = bookmakers_max.rename({0 : 'H', 1 : 'D', 2: 'A'}, axis = 1)
bookmakers_max

bookmakers_max['Prob_bk_H'] = (1/bookmakers_max['H']).round(3)
bookmakers_max['Prob_bk_D'] = (1/bookmakers_max['D']).round(3)
bookmakers_max['Prob_bk_A'] = (1/bookmakers_max['A']).round(3)

In [206]:
#fusion bookmakers et stats
df_1617 = df_1617.replace(['West Bromwich Albion','Manchester City', 'Manchester United'], ['West Brom','Man City', 'Man United'])
df = bookmakers_max.merge(right = df_1617.drop(['date', 'id_match','team_id_home', 'team_id_away', 'Home_wins', 'Draw', 'Away_wins'], axis = 1), on = ['team_name_home', 'team_name_away'])
#date en datetime
df['date'] = pd.to_datetime(df['date'])


In [128]:
#rajouter possession depuis le début
df[['possession_h_moy', 'possession_a_moy']] = 0.0

for i in range(df.shape[0]):
    home = df.loc[i]['team_name_home']
    away = df.loc[i]['team_name_away']
    possession_h_moy = []
    possession_a_moy = []
    for j in range(i):
        if df.loc[j]['team_name_home'] == home:
            possession_h_moy.append(df.loc[j]['possession_percentage_x'])
        if df.loc[j]['team_name_away'] == home:
            possession_h_moy.append(df.loc[j]['possession_percentage_y'])
        if df.loc[j]['team_name_home'] == away:
            possession_a_moy.append(df.loc[j]['possession_percentage_x'])
        if df.loc[j]['team_name_away'] == away:
            possession_h_moy.append(df.loc[j]['possession_percentage_y'])
    if len(possession_h_moy) != 0 and len(possession_a_moy) != 0:
        df.at[i,'possession_h_moy'] = (sum(possession_h_moy) / len(possession_h_moy)).round(2)
        df.at[i,'possession_a_moy'] = (sum(possession_a_moy) / len(possession_a_moy)).round(2)
        


In [16]:
#nombre de points depuis le début et 5 derniers matchs
df[['points_h', 'points_a']] = 0
df[['points_h_last_5', 'points_a_last_5']] = 0
for i in range(df.shape[0]):
    home = df.loc[i]['team_name_home']
    away = df.loc[i]['team_name_away']
    points_home = []
    points_away = []
    for j in range(i):
        if df.loc[j]['team_name_home'] == home:
            points_home.append(3 if df.loc[j]['FTR'] == 'H' else 1 if df.loc[j]['FTR'] == 'D' else 0)
        if df.loc[j]['team_name_away'] == home:
            points_home.append(3 if df.loc[j]['FTR'] == 'A' else 1 if df.loc[j]['FTR'] == 'D' else 0)
        if df.loc[j]['team_name_home'] == away:
            points_away.append(3 if df.loc[j]['FTR'] == 'H' else 1 if df.loc[j]['FTR'] == 'D' else 0)
        if df.loc[j]['team_name_away'] == away:
            points_away.append(3 if df.loc[j]['FTR'] == 'A' else 1 if df.loc[j]['FTR'] == 'D' else 0)
        df.at[i,'points_h'] = sum(points_home)
        df.at[i,'points_a'] = sum(points_away)
        df.at[i,'points_h_last_5'] = sum(points_home[-5:])
        df.at[i,'points_a_last_5'] = sum(points_away[-5:])

In [17]:
#nombre de goals marqués depuis le début
df[['total_goals_h', 'total_goals_a']] = 0
for i in range(df.shape[0]):
    home = df.loc[i]['team_name_home']
    away = df.loc[i]['team_name_away']
    goals_home = []
    goals_away = []
    for j in range(i):
        if df.loc[j]['team_name_home'] == home:
            goals_home.append(df.loc[j]['goals_x'])
        if df.loc[j]['team_name_away'] == home:
            goals_home.append(df.loc[j]['goals_y'])
        if df.loc[j]['team_name_home'] == away:
            goals_away.append(df.loc[j]['goals_x'])
        if df.loc[j]['team_name_away'] == away:
            goals_away.append(df.loc[j]['goals_y'])
        df.at[i,'total_goals_h'] = sum(goals_home)
        df.at[i,'total_goals_a'] = sum(goals_away)

In [18]:
#nombre de goals encaissés depuis le début
df[['total_goals_taken_h', 'total_goals_taken_a']] = 0
for i in range(df.shape[0]):
    home = df.loc[i]['team_name_home']
    away = df.loc[i]['team_name_away']
    goals_taken_home = []
    goals_taken_away = []
    for j in range(i):
        if df.loc[j]['team_name_home'] == home:
            goals_taken_home.append(df.loc[j]['goals_y'])
        if df.loc[j]['team_name_away'] == home:
            goals_taken_home.append(df.loc[j]['goals_x'])
        if df.loc[j]['team_name_home'] == away:
            goals_taken_away.append(df.loc[j]['goals_y'])
        if df.loc[j]['team_name_away'] == away:
            goals_taken_away.append(df.loc[j]['goals_x'])
        df.at[i,'total_goals_taken_h'] = sum(goals_taken_home)
        df.at[i,'total_goals_taken_a'] = sum(goals_taken_away)

In [19]:
#nombre de tirs cadrés depuis le début
df[['total_ontarget_h', 'total_ontarget_a']] = 0
for i in range(df.shape[0]):
    home = df.loc[i]['team_name_home']
    away = df.loc[i]['team_name_away']
    ontarget_home = []
    ontarget_away = []
    for j in range(i):
        if df.loc[j]['team_name_home'] == home:
            ontarget_home.append(df.loc[j]['ontarget_scoring_att_x'])
        if df.loc[j]['team_name_away'] == home:
            ontarget_home.append(df.loc[j]['ontarget_scoring_att_y'])
        if df.loc[j]['team_name_home'] == away:
            ontarget_away.append(df.loc[j]['ontarget_scoring_att_x'])
        if df.loc[j]['team_name_away'] == away:
            ontarget_away.append(df.loc[j]['ontarget_scoring_att_y'])
        df.at[i,'total_ontarget_h'] = sum(ontarget_home)
        df.at[i,'total_ontarget_a'] = sum(ontarget_away)

In [20]:
#nombre de tirs hors-cadre depuis le début
df[['total_shot_off_target_h', 'total_shot_off_target_a']] = 0
for i in range(df.shape[0]):
    home = df.loc[i]['team_name_home']
    away = df.loc[i]['team_name_away']
    offtarget_home = []
    offtarget_away = []
    for j in range(i):
        if df.loc[j]['team_name_home'] == home:
            offtarget_home.append(df.loc[j]['shot_off_target_x'])
        if df.loc[j]['team_name_away'] == home:
            offtarget_home.append(df.loc[j]['shot_off_target_y'])
        if df.loc[j]['team_name_home'] == away:
            offtarget_away.append(df.loc[j]['shot_off_target_x'])
        if df.loc[j]['team_name_away'] == away:
            offtarget_home.append(df.loc[j]['shot_off_target_y'])
        df.at[i,'total_shot_off_target_h'] = sum(offtarget_home)
        df.at[i,'total_shot_off_target_a'] = sum(offtarget_away)

In [21]:
#nombre de tirs bloqués depuis le début
df[['total_blocked_scoring_att_h','total_blocked_scoring_att_a']] = 0
for i in range(df.shape[0]):
    home = df.loc[i]['team_name_home']
    away = df.loc[i]['team_name_away']
    blocked_scoring_home = []
    blocked_scoring_away = []
    for j in range(i):
        if df.loc[j]['team_name_home'] == home:
            blocked_scoring_home.append(df.loc[j]['blocked_scoring_att_x'])
        if df.loc[j]['team_name_away'] == home:
            blocked_scoring_home.append(df.loc[j]['blocked_scoring_att_y'])
        if df.loc[j]['team_name_home'] == away:
            blocked_scoring_away.append(df.loc[j]['blocked_scoring_att_x'])
        if df.loc[j]['team_name_away'] == away:
            blocked_scoring_away.append(df.loc[j]['blocked_scoring_att_y'])
        df.at[i,'total_blocked_scoring_att_h'] = sum(blocked_scoring_home)
        df.at[i,'total_blocked_scoring_att_a'] = sum(blocked_scoring_away)

In [130]:
#note moyenne depuis le début
df[['mean_rating_home','mean_rating_away']] = 0.0
for i in range(df.shape[0]):
    home = df.loc[i]['team_name_home']
    away = df.loc[i]['team_name_away']
    rating_home = []
    rating_away = []
    for j in range(i):
        if df.loc[j]['team_name_home'] == home:
            rating_home.append(df.loc[j]['team_rating_home'])
        if df.loc[j]['team_name_away'] == home:
            rating_home.append(df.loc[j]['team_rating_away'])
        if df.loc[j]['team_name_home'] == away:
            rating_away.append(df.loc[j]['team_rating_home'])
        if df.loc[j]['team_name_away'] == away:
            rating_away.append(df.loc[j]['team_rating_away'])
    if len(rating_home) != 0 and len(rating_away) != 0:
        df.at[i,'mean_rating_home'] = (sum(rating_home) / len(rating_home)).round(2)
        df.at[i,'mean_rating_away'] = (sum(rating_away) / len(rating_away)).round(2)

In [191]:
df = df.drop(['team_rating_home',
              'team_rating_away',
              'accurate_pass_x',
             'accurate_pass_y',
             'blocked_scoring_att_x',
             'blocked_scoring_att_y',
             'goals_x',
             'goals_y',
             'ontarget_scoring_att_x',
             'ontarget_scoring_att_y',
             'possession_percentage_x',
             'possession_percentage_y',
             'post_scoring_att_x',
             'post_scoring_att_y',
             'shot_off_target_x',
             'shot_off_target_y',
             'won_contest_x',
             'won_contest_y',
             'won_corners_x',
             'won_corners_y'],axis=1)

KeyError: "['team_rating_home' 'team_rating_away' 'accurate_pass_x' 'accurate_pass_y'\n 'blocked_scoring_att_x' 'blocked_scoring_att_y' 'goals_x' 'goals_y'\n 'ontarget_scoring_att_x' 'ontarget_scoring_att_y'\n 'possession_percentage_x' 'possession_percentage_y' 'post_scoring_att_x'\n 'post_scoring_att_y' 'shot_off_target_x' 'shot_off_target_y'\n 'won_contest_x' 'won_contest_y' 'won_corners_x' 'won_corners_y'] not found in axis"

In [141]:
df = df.rename({'H': 'bk_H', 'D': 'bk_D', 'A':'bk_A'}, axis = 1)

#dichotomisation des équipes
df = pd.get_dummies(df, prefix = ['H', 'A'], columns = ['team_name_home', 'team_name_away'])
#dichotomisation des résultats W-D-L
#df = pd.get_dummies(df, prefix = '', prefix_sep = '', columns = ['FTR'])

#numérisation de target
df['FTR'] = df.replace(['H','D','A'],['0','1','2'])

KeyError: "None of [Index(['team_name_home', 'team_name_away'], dtype='object')] are in the [columns]"

In [6]:
#import du fichier csv sauvegardé pour éviter de refaire tout le nettoyage
df = pd.read_csv('bookmakers.csv', index_col = 0)
df.head()

,date,bk_H,bk_D,bk_A,Prob_bk_H,Prob_bk_D,Prob_bk_A,possession_h_moy,possession_a_moy,points_h,points_a,points_h_last_5,points_a_last_5,total_goals_h,total_goals_a,total_goals_taken_h,total_goals_taken_a,total_ontarget_h,total_ontarget_a,total_shot_off_target_h,total_shot_off_target_a,total_blocked_scoring_att_h,total_blocked_scoring_att_a,mean_rating_home,mean_rating_away,H_Arsenal,H_Bournemouth,H_Burnley,H_Chelsea,H_Crystal Palace,H_Everton,H_Hull,H_Leicester,H_Liverpool,H_Man City,H_Man United,H_Middlesbrough,H_Southampton,H_Stoke,H_Sunderland,H_Swansea,H_Tottenham,H_Watford,H_West Brom,H_West Ham,A_Arsenal,A_Bournemouth,A_Burnley,A_Chelsea,A_Crystal Palace,A_Everton,A_Hull,A_Leicester,A_Liverpool,A_Man City,A_Man United,A_Middlesbrough,A_Southampton,A_Stoke,A_Sunderland,A_Swansea,A_Tottenham,A_Watford,A_West Brom,A_West Ham,FTR
0,2016-08-13,2.79,3.32,3.25,0.358,0.301,0.308,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2
1,2016-08-13,2.25,3.30,4.50,0.444,0.303,0.222,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2
2,2016-08-13,3.64,3.54,2.50,0.275,0.282,0.400,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,2016-08-13,4.68,3.60,2.10,0.214,0.278,0.476,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2016-08-13,1.27,6.50,15.00,0.787,0.154,0.067,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [100]:
#Première itération

# Séparation du jeu de données en variables explicatives et variable cible.
features = df.drop(['date','bk_H','bk_D','bk_A','Prob_bk_H','Prob_bk_D','Prob_bk_A','FTR'],axis=1)

target = df['FTR']



# Train test split en gardant l'ordre des matchs
X_train, X_test, y_train, y_test = train_test_split(features,
                                                    target,
                                                    test_size = 0.2,
                                                    shuffle=False,
                                                    random_state = 777)


In [103]:
#standardisation
scaler = preprocessing.StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [104]:
#première itération svm
clf = SVC(gamma=0.01, probability = True)
clf.fit(X_train_scaled, y_train)

y_train_pred = clf.predict(X_train_scaled)
print(pd.crosstab(y_train, y_train_pred, rownames=['Classe réelle'], colnames=['Classe prédite']))
accuracy_score_train = accuracy_score(y_train, y_train_pred)
print("accuracy score test : ",accuracy_score_train)



y_pred = clf.predict(X_test_scaled)
print(pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite']))
accuracy_score_test = accuracy_score(y_test, y_pred)
print("accuracy score test : ",accuracy_score_test)

Classe prédite    0  1   2
Classe réelle             
0               146  1   4
1                51  9  10
2                33  0  50
accuracy score test :  0.6743421052631579
Classe prédite   0  1   2
Classe réelle            
0               33  1   2
1               11  1   2
2               15  0  11
accuracy score test :  0.5921052631578947


In [105]:
#optimisation des hyperparamètres à l'aide d'une grille
parametres = {'C':[0.1,1,10], 'kernel':['rbf','linear', 'poly'], 'gamma':[0.001, 0.1, 0.5]}
grid_clf = model_selection.GridSearchCV(estimator=clf, param_grid=parametres)
grille = grid_clf.fit(X_train_scaled,y_train)
print(pd.DataFrame.from_dict(grille.cv_results_).loc[:,['params', 'mean_test_score']]) 
print(grid_clf.best_params_)
y_pred = grid_clf.predict(X_test_scaled)
pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])

                                            params  mean_test_score
0      {'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}         0.496721
1   {'C': 0.1, 'gamma': 0.001, 'kernel': 'linear'}         0.542787
2     {'C': 0.1, 'gamma': 0.001, 'kernel': 'poly'}         0.496721
3        {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}         0.496721
4     {'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}         0.542787
5       {'C': 0.1, 'gamma': 0.1, 'kernel': 'poly'}         0.546120
6        {'C': 0.1, 'gamma': 0.5, 'kernel': 'rbf'}         0.496721
7     {'C': 0.1, 'gamma': 0.5, 'kernel': 'linear'}         0.542787
8       {'C': 0.1, 'gamma': 0.5, 'kernel': 'poly'}         0.546120
9        {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}         0.496721
10    {'C': 1, 'gamma': 0.001, 'kernel': 'linear'}         0.506448
11      {'C': 1, 'gamma': 0.001, 'kernel': 'poly'}         0.496721
12         {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}         0.496721
13      {'C': 1, 'gamma': 0.1, 'kernel': 'linear

Classe prédite,0,1,2
Classe réelle,,,
0,32,2,2
1,11,1,2
2,12,1,13


In [106]:
#détermination des probabilités par le modèle svm
y_pred_proba = grid_clf.predict_proba(X_test_scaled)

In [107]:
#création des colonnes de probas déterminés par le modèle
df['Prob_svm_H'] = 0.0
df['Prob_svm_D'] = 0.0
df['Prob_svm_A'] = 0.0

df.loc[304:]['Prob_svm_H'] = y_pred_proba[:,0]
df.loc[304:]['Prob_svm_D'] = y_pred_proba[:,1]
df.loc[304:]['Prob_svm_A'] = y_pred_proba[:,2]

<ipython-input-107-48583fb0dac6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[304:]['Prob_svm_H'] = y_pred_proba[:,0]
<ipython-input-107-48583fb0dac6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[304:]['Prob_svm_D'] = y_pred_proba[:,1]
<ipython-input-107-48583fb0dac6>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [129]:
#détermination des paris intéréssants

df.loc[df['Prob_svm_H']>df['Prob_bk_H'],'bet_H?'] = 1
df.loc[df['Prob_svm_H']<df['Prob_bk_H'],'bet_H?'] = 0
df.loc[df['Prob_svm_D']>df['Prob_bk_D'],'bet_D?'] = 1
df.loc[df['Prob_svm_D']<df['Prob_bk_D'],'bet_D?'] = 0
df.loc[df['Prob_svm_A']>df['Prob_bk_A'],'bet_A?'] = 1
df.loc[df['Prob_svm_A']<df['Prob_bk_A'],'bet_A?'] = 0
df = df.fillna(0)

In [130]:

matchs = []
for i in range(380):
    matchs.extend([i,i,i])

resultats = []
for i in range(380):
    resultats.extend([0,1,2])  
montants = []
for i in range(380):
    montants.extend(df.loc[i][['bet_H?', 'bet_D?', 'bet_A?']])

In [78]:
# Simulation d'un pari
def gain_paris(matchs, resultats, montants):
    benefices = 0
    for match, resultat, montant in zip(matchs, resultats, montants):
        if df.loc[match]['FTR'] == resultat:
            benefice = (df.loc[match][resultat+1] - 1)*montant
        else : benefice = -1*montant
        benefices += benefice
    return benefices.round(2)

In [131]:
# Évaluation du modèle
gain_clf = gain_paris(matchs, resultats, montants)

print("Gain/perte des paris avec le modele clf:", gain_clf)

Gain/perte des paris avec le modele clf: -21.25


In [120]:
#deuxième itération

rf = RandomForestClassifier(n_estimators = 1000,
                            max_depth=4,
                            n_jobs=-1)

rf_pipe = Pipeline(steps=[('normalisation',sc),
                         ('model',rf)])

rf_pipe.fit(X_train_scaled,y_train)

y_pred = rf_pipe.predict(X_test_scaled) # 76 lignes

In [121]:
y_train_predict = rf_pipe.predict(X_train_scaled)

print(pd.crosstab(y_train, y_train_predict, rownames=['Classe réelle'], colnames=['Classe prédite']))
print("Train Accuracy score: ", accuracy_score(y_train, y_train_predict))


print(pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite']))
print("Test Accuracy score: ", accuracy_score(y_test, y_pred))

Classe prédite    0  1   2
Classe réelle             
0               146  0   5
1                52  6  12
2                29  0  54
Train Accuracy score:  0.6776315789473685
Classe prédite   0  1   2
Classe réelle            
0               33  1   2
1               13  0   1
2               13  0  13
Test Accuracy score:  0.6052631578947368


In [122]:
#prédiction des probas
y_pred_proba = rf_pipe.predict_proba(X_test_scaled)

In [126]:
#création des colonnes de probas déterminés par le modèle
df['Prob_rf_H'] = 0.0
df['Prob_rf_D'] = 0.0
df['Prob_rf_A'] = 0.0

df.loc[304:]['Prob_rf_H'] = y_pred_proba[:,0]
df.loc[304:]['Prob_rf_D'] = y_pred_proba[:,1]
df.loc[304:]['Prob_rf_A'] = y_pred_proba[:,2]

#détermination des paris intéréssants


df.loc[df['Prob_rf_H']>df['Prob_bk_H'],'bet_H?'] = 1
df.loc[df['Prob_rf_H']<df['Prob_bk_H'],'bet_H?'] = 0
df.loc[df['Prob_rf_D']>df['Prob_bk_D'],'bet_D?'] = 1
df.loc[df['Prob_rf_D']<df['Prob_bk_D'],'bet_D?'] = 0
df.loc[df['Prob_rf_A']>df['Prob_bk_A'],'bet_A?'] = 1
df.loc[df['Prob_rf_A']<df['Prob_bk_A'],'bet_A?'] = 0
df = df.fillna(0)

<ipython-input-126-bcb519dfb10b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[304:]['Prob_rf_H'] = y_pred_proba[:,0]
<ipython-input-126-bcb519dfb10b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[304:]['Prob_rf_D'] = y_pred_proba[:,1]
<ipython-input-126-bcb519dfb10b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [127]:
matchs = []
for i in range(380):
    matchs.extend([i,i,i])

resultats = []
for i in range(380):
    resultats.extend([0,1,2])  
montants = []
for i in range(380):
    montants.extend(df.loc[i][['bet_H?', 'bet_D?', 'bet_A?']])

In [128]:
gain_rf = gain_paris(matchs, resultats, montants)
print("Gain/perte des paris avec le modele rd_pipe:", gain_rf)

Gain/perte des paris avec le modele rd_pipe: -28.14


In [ ]:
# GridSearchCV avec plusieurs modèles et une grille de paramètres (Pipeline)



In [ ]:
# Entrainement du meilleur modèle avec les meilleurs paramètres



In [ ]:
# Évaluation du meilleur modèle

